## 오차 역전파의 계산법

은닉층의 노드마다 x -> sum(w * x + b) = z -> activation(z) = y -> y

W_31(t + 1) = W_31(t) - (loss에 대한 W_31 편미분계수)

- 오차 역전파이므로 출력층에서부터 계산
- 오차, loss = 평균 오차 제곱합

연쇄 법칙, chain rule
- 하나의 가중치에 대한 편미분계수를 계산하려면, 이후 거치는 모든 연산에 대한 미분계수가 필요하다.<br />(합성함수의 미분계수는 각 계산의 미분계수를 곱해야 한다)

<pre>
bias는 graph를 좌우로 움직이는 역할을 하지.
활성화 함수로 사용되는 sigmoid가 가장 안정된 예측을 하게 하는 bias 값이 1이다.
무엇보다 미분에서 상수항은 사라지기 때문에 근본적으로 어떤 값이든 상관이 없다.
</pre>

출력층이라 다른 class의 loss를 고려할 필요가 업다면
- *W_31(t + 1) = W_31(t) - (y - yt) * y * (1 - y) * x*
- 대게의 경우, 편미분에서 class별 loss가 사라지지 않아 직접하지 않고, 앞에 식에 이번 편미분계수를 곱하는 것으로 한다.

가중치 update를 보다 빠르게 연산하고자 node마다 매 udpate step에서 동일하게 사용되는 구간을 delta 식으로 기억해둔다.<br />(이때 delta 식은 out * (1 - out)의 형태)

-> W_11(t + 1) = W_11(t) - delta_h * x

## 파이썬 코드로 확인하는 신경망

In [26]:
import numpy as np
import random

### custom

In [27]:
data = [[[0, 0], [0]], [[0, 1], [1]], [[1, 0], [1]], [[1, 1], [0]]]

In [28]:
lr = 0.1
mo = 0.4
iterations = 5000

In [29]:
random.seed(777)

### custom function

In [30]:
def makeMatrix(i, j, fill=0.0):
    mat = []
    for i in range(i):
        mat.append([fill] * j)
    return mat

In [31]:
def sigmoid(x, derivative=False):
    if derivative == True:
        return x * (1 - x)
    return 1 / (1 + np.exp(-x))

In [32]:
def tanh(x, derivative=False):
    if derivative == True:
        return 1 - x ** 2
    return np.tanh(x)

### model

초깃값 지정 -> 업데이트 함수 -> 역전파 함수

In [33]:
class NeuralNetwork:
    def __init__(self, num_x, num_yh, num_y0, bias=1):
        # variables
        self.num_x = num_x + bias
        self.num_yh = num_yh
        self.num_y0 = num_y0

        # activation function
        self.activation_input = [1.0] * self.num_x
        self.activation_hidden = [1.0] * self.num_yh
        self.activation_out = [1.0] * self.num_y0

        # weights
        self.weight_in = makeMatrix(self.num_x, self.num_yh)
        for i in range(self.num_x):
            for j in range(self.num_yh):
                self.weight_in[i][j] = random.random()

        self.weight_out = makeMatrix(self.num_yh, self.num_y0)
        for i in range(self.num_yh):
            for j in range(self.num_y0):
                self.weight_out[i][j] = random.random()

        # before weight init for momentum SGD
        self.gradient_in = makeMatrix(self.num_x, self.num_yh)
        self.gradient_out = makeMatrix(self.num_yh, self.num_y0)

    # https://goo.gl/f6khsU 참조
    def update(self, inputs):
        # 입력층 activation function
        for i in range(self.num_x - 1):
            self.activation_input[i] = inputs[i]

        # 은닉층 activation function
        for i in range(self.num_yh):
            sum = 0.0
            for j in range(self.num_x):
                sum += self.activation_input[j] * self.weight_in[j][i]
            self.activation_hidden[i] = tanh(sum, False)

        # 출력층 activation function
        for i in range(self.num_y0):
            sum = 0.0
            for j in range(self.num_yh):
                sum += self.activation_hidden[j] * self.weight_out[j][i]
            self.activation_out[i] = tanh(sum, False)
        return self.activation_out[:]

    # momentum back-propagation
    def backPropagate(self, targets):
        # 출력층 delta식
        output_deltas = [0.0] * self.num_y0
        for k in range(self.num_y0):
            error = targets[k] - self.activation_out[k]
            output_deltas[k] = tanh(self.activation_out[k],  True) * error

        # 은닉층 delta식
        hidden_deltas = [0.0] * self.num_yh
        for j in range(self.num_yh):
            error = 0.0
            for k in range(self.num_y0):
                error += output_deltas[k] * self.weight_out[j][k]
            hidden_deltas[j] = tanh(self.activation_hidden[j], True) * error

        # 출력층 update
        for j in range(self.num_yh):
            for k in range(self.num_y0):
                gradient = output_deltas[k] * self.activation_hidden[j]
                v = mo * self.gradient_out[j][k] - lr * gradient
                self.weight_out[j][k] += v
                self.gradient_out[j][k] = gradient

        # 입력층 update
        for i in range(self.num_x):
            for j in range(self.num_yh):
                gradient = hidden_deltas[j] * self.activation_input[i]
                v = mo * self.gradient_in[i][j] - lr * gradient
                self.weight_in[i][j] += v
                self.gradient_in[i][j] = gradient

        error = 0.0
        for k in range(len(targets)):
            error += 0.5 * (targets[k] - self.activation_out[k]) ** 2
        return error

    def train(self, iterations, patterns):
        for i in range(iterations):
            error = 0.0
            for p in patterns:
                inputs = p[0]
                targets = p[1]
                self.update(inputs)
                error += self.backPropagate(targets)

            if i % 500 == 0:
                print("error: %-.5f" % error)

    def result(self, patterns):
        for p in patterns:
            print("Input: %s, Predict: %s" % (p[0], self.update(p[0])))

In [34]:
if __name__ == "__main__":
    n = NeuralNetwork(2, 2, 1)
    n.train(iterations, data)
    n.result(data)

error: 0.66537
error: 0.00263
error: 0.00088
error: 0.00051
error: 0.00036
error: 0.00027
error: 0.00022
error: 0.00018
error: 0.00016
error: 0.00014
Input: [0, 0], Predict: [0.0006183430577843577]
Input: [0, 1], Predict: [0.9889696478602484]
Input: [1, 0], Predict: [0.9889970505963889]
Input: [1, 1], Predict: [0.0021449252379778148]
